In [1]:
import os

folders = ["data", "vectorstore"]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Project folders created")


Project folders created


In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/sample_ml_notes.pdf")
documents = loader.load()

print("Documents loaded:", len(documents))


Documents loaded: 5


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)

print("Total chunks:", len(chunks))
print(chunks[0].page_content[:200])


Total chunks: 5
Introduction to Machine Learning
Machine Learning is a field of Artificial Intelligence that enables systems to learn from data without
being explicitly programmed. Instead of writing rules manually, 


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load a lightweight, industry-standard embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Quick sanity test
vector = embeddings.embed_query("What is supervised learning?")
print("Embedding vector length:", len(vector))


C:\Users\cletu\AppData\Local\Temp\ipykernel_28048\1220627759.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\cletu\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cletu\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding vector length: 384


In [10]:
from langchain_community.vectorstores import FAISS

# Create FAISS index from document chunks
vectorstore = FAISS.from_documents(chunks, embeddings)

# Save FAISS index locally
vectorstore.save_local("vectorstore")

print("FAISS vector store created and saved")


FAISS vector store created and saved


In [12]:
# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Retrieve relevant documents (NEW API)
docs = retriever.invoke("Explain supervised learning")

print("Retrieved text:")
print(docs[0].page_content)


Retrieved text:
Supervised Learning
Supervised learning involves training a model on input-output pairs. Common algorithms include
Linear Regression, Logistic Regression, Decision Trees, and Support Vector Machines.


In [13]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

# Load a lightweight instruction-tuned model
hf_pipeline = pipeline(
    task="text2text-generation",
    model="google/flan-t5-base",
    max_length=512
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

print("LLM loaded successfully")


config.json: 0.00B [00:00, ?B/s]

c:\Users\cletu\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cletu\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For 

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Error while downloading from https://huggingface.co/google/flan-t5-base/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


model.safetensors:   1%|1         | 10.5M/990M [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


LLM loaded successfully


C:\Users\cletu\AppData\Local\Temp\ipykernel_28048\2606843425.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [15]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
You are a helpful AI assistant.

Answer the question ONLY using the context provided below.
If the answer is NOT present in the context, say exactly:
"I don't know based on the provided documents."

Context:
{context}

Question:
{question}

Answer:
""",
    input_variables=["context", "question"]
)


In [19]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [20]:
rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)



In [21]:
rag_chain.invoke("What is supervised learning?")



Token indices sequence length is longer than the specified maximum sequence length for this model (667 > 512). Running this sequence through the model will result in indexing errors


"training a model on input-output pairs. Common algorithms includenLinear Regression, Logistic Regression, Decision Trees, and Support Vector Machines.')"

In [22]:
rag_chain.invoke("Explain quantum computing")


"I don't know"